In [ ]:
import os
import sys
sys.path.append(os.path.join('..', '..'))

import logging
logging.getLogger().setLevel(logging.INFO)

import datetime
import pandas as pd
import multiprocessing as mp

from bokeh.io import show, output_notebook
from bokeh.models import Panel, Tabs, Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, ColumnDataSource, LabelSet, PointDrawTool, WheelZoomTool
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.models.widgets import Div
from bokeh.plotting import figure
from bokeh.layouts import layout
from bokeh.models.tools import HoverTool
import networkx as nx

# custom libraries
from mikesnowflake.analysis.snowFlakeAnalysis import SnowFlakeAnalysis
from mikesnowflake.access.colorAccess import ColorAccess

output_notebook(hide_banner=True)

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ''

user = ''
password = ''

In [ ]:
START_DATE = datetime.datetime(2019, 12, 1)
END_DATE = datetime.datetime(2020, 6, 26)
SFA = SnowFlakeAnalysis(START_DATE, END_DATE, user, password, excludeEtl=False, verbose=False)
ca = ColorAccess()

In [ ]:
def getHistory(tableNames, processes=8, sfa=SFA):
    """
    """
    startTime = datetime.datetime.today()
    if isinstance(tableNames, str):
        tableNames = [tableNames]
    with mp.Pool(processes) as pool:
        out = pool.map(sfa.getQueryTypeHistory, tableNames)
    endTime = datetime.datetime.today()
    return dict(zip(tableNames, out))

In [ ]:
tableNames = ['SUPPLY_DEMAND_GEO_HOURLY_FACT',
 'PLATFORM_DIM',
 'SUPPLY_DEMAND_COUNTRY_HOURLY_FACT',
 'ADVERTISER_DIM',
 'PUBLISHER_DIM',
 'TIMEZONE',
 'DEMAND_HOURLY_FACT',
 'LINE_ITEM_DIM',
 'SUPPLY_HOURLY_FACT',
 'AD_UNIT_DIM',
 'CONTENT_TOPIC_GROUP_DIM',
 'SUPPLY_DEMAND_PAIRS_FACT',
 'SUPPLY_SIZE_FACT',
 'AD_UNIT_DEAL_TYPE_HISTORY_NEW',
 'CPD_LINE_ITEM_HISTORY_NEW',
 'ROLLUP_STATE',
 'DATA_CHANGE']
qtHistory = getHistory(SFA.snowFlakeTables, processes=8)

In [ ]:
# show overall query type history for tables in question
p = {}  # collector of bokeh charts keyed by table name
for tableName in tableNames:
    tableHistory = qtHistory[tableName].pivot(index='query_date', columns='query_type', values='hits')
    tableHistory.columns.name = None
    
    p[tableName] = figure(width=1900, height=800, x_axis_type='datetime')
    for c in tableHistory.columns:
        p[tableName].line(tableHistory.index.tolist(), tableHistory[c], line_width=2, name=c, legend_label=c, line_color=SFA.queryTypeColors[c])
    p[tableName].title.text = 'Daily SQL Command Count History for %s' % tableName
    p[tableName].title.align = 'center'
    hover = HoverTool(tooltips=[('Date', '@x{%F}'),
                                ("Query Type", "$name"),
                                ("Hits", "@y{0,0}")],
                      formatters={'x': 'datetime'})
    p[tableName].add_tools(hover)
    p[tableName].legend.click_policy="hide"
tabSize = 5
subLists = [tableNames[i * tabSize:(i + 1) * tabSize] for i in range((len(tableNames) + tabSize - 1) // tabSize )]
tabs = []
for subList in subLists:
    tabs.append([Tabs(tabs=[Panel(child=p[tableName], title=tableName) for tableName in subList], background='whitesmoke')])
show(layout([Div(text="Daily SQL Command Count History </b>", style={'font-size': '150%'}, width=1500, height=25)],
            [tabs]))

In [ ]:
# show select-type usage
for tableName in tableNames:
    df = SFA.getUsageHistory(tableName, 'select')
    colors = dict(zip(df['user_name'].unique().tolist(), ca.getColors(len(df['user_name'].unique()))))
    q = {}  # collector of bokeh charts keyed by table name
    for queryCmd, out in df.groupby(['query_type']):
        userHistory = out.pivot(index='query_date', columns='user_name', values='hits')
        userHistory.columns.name = None
        userHistory = userHistory.reindex(pd.date_range(SFA.startDate, SFA.endDate)).fillna(0)

        q[queryCmd] = figure(width=1900, height=1000, x_axis_type='datetime')
        for user in userHistory.columns:
            q[queryCmd].line(userHistory.index.tolist(), userHistory[user], line_width=2, name=user, legend_label=user, line_color=colors[user])
        q[queryCmd].title.text = '%s History (%s)' % (tableName, queryCmd)
        q[queryCmd].title.align = 'center'
        hover = HoverTool(tooltips=[('Date', '@x{%F}'),
                                    ("User", "$name"),
                                    ("Hits", "@y{0,0}")],
                          formatters={'x': 'datetime'})
        q[queryCmd].add_tools(hover)
        q[queryCmd].legend.click_policy="hide"

    tabSize = 5
    queryTypes = list(q.keys())
    subLists = [queryTypes[i * tabSize:(i + 1) * tabSize] for i in range((len(queryTypes) + tabSize - 1) // tabSize )]
    tabs = []
    for subList in subLists:
        tabs.append([Tabs(tabs=[Panel(child=q[queryType], title=queryType) for queryType in subList], background='whitesmoke')])

    for tab in tabs:
        show(layout([Div(text="Select-like Command User History for %s</b>" % tableName, style={'font-size': '150%'}, width=1500, height=25)],
                    [tab]))

In [ ]:
G = SFA.tableGraph
edges = []
for n in tableNames:
    nodeEdges = G.edges(n)
    if len(nodeEdges) > 0:
        for e in nodeEdges:
            edges.append(e)
    elif n in SFA.snowFlakeViews:
        for (e1, e2) in G.edges():
            if e2 == n:
                edges.append((e1, e2))
H = nx.DiGraph(edges)
colorMap = {n: {'color': 'red'} if n in tableNames else {'color': ca.getNodeDefaultColor()} for n in H.nodes()}  # highlight the node as red
title = 'Table Dependency Graph'
nx.set_node_attributes(H, colorMap)
    
plot = figure(title=title, x_range=(-1.1,1.1), y_range=(-1.1,1.1), height=1000, width=1500, tools="pan,box_zoom,reset")
plot.title.align = 'center'

graph_renderer = from_networkx(H, nx.spring_layout, scale=1, center=(0,0))
graph_renderer.node_renderer.glyph = Circle(size=15, fill_color='color')
graph_renderer.node_renderer.selection_glyph = Circle(size=15, fill_color=ca.getNodeSelectColor())
graph_renderer.node_renderer.hover_glyph = Circle(size=15, fill_color=ca.getNodeHoverColor())

graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=1, line_width=1)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=ca.getNodeSelectColor(), line_width=3)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=ca.getNodeHoverColor(), line_width=3)

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = EdgesAndLinkedNodes()

wheel_zoom = WheelZoomTool()
plot.add_tools(HoverTool(tooltips=None), TapTool(), wheel_zoom)
plot.toolbar.active_scroll=wheel_zoom
plot.renderers.append(graph_renderer)
pos = graph_renderer.layout_provider.graph_layout
x,y=zip(*pos.values())
source = ColumnDataSource({'x':x,'y':y, 'label': list(pos.keys())})
labels = LabelSet(x='x', y='y', text='label', source=source, text_font_size='11px')
plot.renderers.append(labels)
show(plot)

In [ ]:
%%html
<style>
div.input {
    display:none;
}
</style>